In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import warnings

In [2]:
train = pd.read_csv('C:\\Users\\guilh_000\Documents\\tweetsclassify\\train_tweet.csv')
test = pd.read_csv('C:\\Users\\guilh_000\\Documents\\tweetsclassify\\test_tweets.csv')

print(train.shape)
print(test.shape)

(31962, 3)
(17197, 2)


In [3]:
train.isnull().any()
test.isnull().any()

id       False
tweet    False
dtype: bool

In [5]:
from textblob import TextBlob
def lemmatize_with_postag(sentence):
    sent = TextBlob(sentence)
    tag_dict = {"J":'a',"N":'n',"V":'v',"R":'r'}
    words_and_tags = [(w, tag_dict.get(pos[0],'n'))for w, pos in sent.tags]
    lemmatized_list = [wd.lemmatize(tag)for wd, tag in words_and_tags]
    return " ".join(lemmatized_list)

import string
import nltk
from nltk.corpus import stopwords
import re
def normalizer(tweet):
    punct = list(string.punctuation)
    stop_words = stopwords.words('english')
    additional_stop_words = ['RT','rt','via','...','http','twitpic','tinyurl','www','amp']
    stopword_list = punct + stop_words + additional_stop_words
    
    tweet = re.sub("(@[A-Za-z0-9]+)|(#[A-Za-z0-9]+)", " ", tweet)
    tweet_ = re.sub("(\w+:\/\/\S+)", " ", tweet)
    tweet__ = re.sub("[^a-zA-Z]", " ", tweet_)
    lemmatized = lemmatize_with_postag(tweet__)
    tokens = nltk.word_tokenize(lemmatized)[2:]
    lower_case = [l.lower() for l in tokens]
    filtered_result = list(filter(lambda l: l not in stopword_list, lower_case))
    return filtered_result

In [6]:
train['normalized'] = train.tweet.apply(normalizer)
train['normal'] = [' '.join(map(str, l)) for l in train['normalized']]
train.head(10)

,id,label,tweet,normalized,normal
0,1,0,@user when a father is dysfunctional and is s...,"[father, dysfunctional, selfish, drag, kid, dy...",father dysfunctional selfish drag kid dysfunction
1,2,0,@user @user thanks for #lyft credit i can't us...,"[credit, use, cause, offer, wheelchair, van, pdx]",credit use cause offer wheelchair van pdx
2,3,0,bihday your majesty,[majesty],majesty
3,4,0,#model i love u take with u all the time in ...,"[u, take, u, time, ur]",u take u time ur
4,5,0,factsguide: society now #motivation,[],
5,6,0,[2/2] huge fan fare and big talking before the...,"[fare, big, talk, leave, chaos, pay, dispute, ...",fare big talk leave chaos pay dispute get
6,7,0,@user camping tomorrow @user @user @user @use...,[danny],danny
7,8,0,the next school year is the year for exams.ð...,"[school, year, year, exam, think]",school year year exam think
8,9,0,we won!!! love the land!!! #allin #cavs #champ...,"[love, land]",love land
9,10,0,@user @user welcome here ! i'm it's so #gr...,[],


In [7]:
from scipy.sparse import hstack
from sklearn.feature_extraction.text import CountVectorizer

count_vectorizer = CountVectorizer(ngram_range=(1,2))
vectorized_data =count_vectorizer.fit_transform(train['normal'])
indexed_data =hstack((np.array(range(0,vectorized_data.shape[0]))[:,None],vectorized_data))

In [8]:
targets = train.iloc[:, 1]

from sklearn.model_selection import train_test_split 
data_train, data_test, targets_train, targets_test =train_test_split(indexed_data, targets, test_size=0.25,random_state=0)
data_train_index = data_train[:,0]
data_train = data_train[:,1:]
data_test_index = data_test[:,0]
data_test = data_test[:,1:]

In [ ]:
from sklearn.ensemble import RandomForestClassifier
mdl = RandomForestClassifier(n_estimators=1000, random_state=0, max_depth=1000)
mdl.fit(data_train, targets_train) 

In [55]:
targets_pred = mdl.predict(data_test)

In [56]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(targets_test,targets_pred))
print(classification_report(targets_test,targets_pred))
print(accuracy_score(targets_test, targets_pred))

[[7460    0]
 [ 526    5]]
              precision    recall  f1-score   support

           0       0.93      1.00      0.97      7460
           1       1.00      0.01      0.02       531

    accuracy                           0.93      7991
   macro avg       0.97      0.50      0.49      7991
weighted avg       0.94      0.93      0.90      7991

0.9341759479414341


In [ ]:
[7379   81] #100 2000
 [ 311  220]]
              precision    recall  f1-score   support

           0       0.96      0.99      0.97      7460
           1       0.73      0.41      0.53       531

    accuracy                           0.95      7991
   macro avg       0.85      0.70      0.75      7991
weighted avg       0.94      0.95      0.94      7991

0.9509448129145288